In [32]:
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio
import cv2
import matplotlib.pyplot as plt

In [28]:
from PIL import Image
import glob
import os
import re
for root, subfolder, filenames in os.walk("./reconstructed_images"):
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    print(filenames)
    for i, file in enumerate(filenames):
        img = Image.open(f"./reconstructed_images/{file}")
        box = (200,180,1328,1328)
        img2 = img.crop(box)
        img2.save(f"img_crop_{i}.png")


['image (1).png', 'image (2).png', 'image (3).png', 'image (4).png', 'image (5).png', 'image (6).png', 'image (7).png', 'image (8).png', 'image (9).png', 'image (10).png', 'image (11).png', 'image (12).png', 'image (13).png', 'image (14).png', 'image (15).png', 'image (16).png', 'image (17).png', 'image (18).png', 'image (19).png', 'image (20).png', 'image (21).png', 'image (22).png', 'image (23).png', 'image (24).png', 'image (25).png', 'image (26).png', 'image (27).png', 'image (28).png', 'image (29).png', 'image (30).png', 'image (31).png', 'image (32).png', 'image (33).png', 'image (34).png', 'image (35).png', 'image (36).png', 'image (37).png', 'image (38).png', 'image (39).png', 'image (40).png', 'image (41).png', 'image (42).png', 'image (43).png', 'image (44).png', 'image (45).png', 'image (46).png', 'image (47).png', 'image (48).png', 'image (49).png', 'image (50).png', 'image (51).png', 'image (52).png', 'image (53).png', 'image (54).png', 'image (55).png', 'image (56).png', 

In [42]:
# Load the spectrogram image using OpenCV
n_fft = 512
hop_length = 4
sr=10000

def reconstruct_audio(image):
    spec_image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    
    # Convert the spectrogram image to a numpy array
    spec_array = np.array(spec_image)
        
    # Denormalise Spectrogram
    spec_array = cv2.normalize(spec_array, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX) #Produces cleaner audio although a lot of frequency content is lost

    # Convert the spectrogram array to a magnitude spectrogram
    spec = librosa.db_to_amplitude(spec_array) 
        
    # Reconstruct the audio signal from the magnitude spectrogram
    y_reconstructed = librosa.feature.inverse.mel_to_audio(spec, sr=sr, n_fft=n_fft, n_iter=1, fmax=sr, hop_length=hop_length)
    
    #return y_reconstructed
    # Save the reconstructed audio signal as a WAV file
    return Audio(y_reconstructed, rate=sr)


In [47]:
import scipy.io.wavfile as wavefile

In [48]:
from PIL import Image
import glob
import os
import re
# Set the path to the folder containing the images
image_folder = './Cropped Images/'
# Set the name of the output GIF file
output_folder = "./output/"

# Get a list of all the image file names in the folder
image_files = glob.glob(image_folder + '*.png')
# Sort the file names alphabetically to ensure they are in the correct order

image_files.sort(key=lambda f: int(re.sub('\D', '', f)))
#image_files.sort(key=os.path.getmtime)
def create_final_audio(folder):
    
        audio = np.array([])
        for i, filename in enumerate(image_files):
            spec_image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        
            spec_array = np.array(spec_image)
 
            spec = librosa.db_to_amplitude(spec_array) 
      
            y_reconstructed = librosa.feature.inverse.mel_to_audio(spec, sr=sr, n_fft=n_fft, n_iter=1, fmax=sr, hop_length=hop_length)
            
          
            audio = np.append(audio, y_reconstructed)
        return audio

       


In [49]:
final_array = create_final_audio(image_files)

In [50]:
Audio(final_array, rate=sr)